In [11]:
# %%
import os
from os import path
from pcdet.utils import common_utils
from pcdet.models import build_network, load_data_to_gpu
from pcdet.datasets import DatasetTemplate
from pcdet.config import cfg, cfg_from_yaml_file
import torch
import numpy as np
import argparse
import glob
from pathlib import Path
from eval_utils import eval_utils

# from tools.visual_utils import visualize_utils as V
ABS_PATH_PREFIX = os.getcwd()


# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
#%%
class Args:
    def __init__(self):
        self.cfg_file = path.join(
            ABS_PATH_PREFIX,  'cfgs/kitti_models/second.yaml')
        self.data_path = path.join(
            ABS_PATH_PREFIX, '../data/kitti/')
        self.ckpt = path.join(ABS_PATH_PREFIX,
                              '../output/kitti_models/second/def/ckpt/checkpoint_epoch_50.pth')
        self.ext = '.bin'
        print("cfg _file  = ", self.cfg_file)


In [27]:

#%%

from pcdet.datasets.kitti.kitti_dataset import KittiDataset
os.chdir(ABS_PATH_PREFIX)
args = Args()

cfg_from_yaml_file(args.cfg_file, cfg)

# args, cfg = parse_config()

print("args = ", args)
print("cfg = ", cfg)

logger = common_utils.create_logger()
logger.info(
    '-----------------Quick Demo of OpenPCDet-------------------------')
demo_dataset = KittiDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
    root_path=Path(args.data_path), logger=logger
)
logger.info(f'Total number of samples: \t{len(demo_dataset)}')

model = build_network(model_cfg=cfg.MODEL, num_class=len(
    cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=True)
model.cuda() 
model.eval()
metric = {
    'gt_num' : 0,
}
for cur_thresh in cfg.MODEL.POST_PROCESSING.RECALL_THRESH_LIST:
    metric['recall_roi_%s' % str(cur_thresh)] = 0
    metric['recall_rcnn_%s' % str(cur_thresh)] = 0
        
with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        if idx == 3:
            break
        logger.info(f'Visualized sample index: \t{idx + 1}')
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        pred_dicts, ret_dict = model.forward(data_dict)
        
        disp_dict = {}
        eval_utils.statistics_info(cfg, ret_dict, metric, disp_dict)
        annos = demo_dataset.generate_prediction_dicts(
            data_dict, pred_dicts, demo_dataset.class_names, None
        )
        # pred_dicts includes:
        #     pred_boxes, N x 7
        #     pred_scores, N
        #     pred_labels
        print(pred_dicts)

        print(data_dict.keys())

#         V.draw_scenes(
#             points=data_dict['points'][:,1:], ref_boxes=pred_dicts[0]['pred_boxes'],
#             ref_scores=pred_dicts[0]['pred_scores'], ref_labels=pred_dicts[0]['pred_labels']
#         )
#         mlab.show(stop=True)


logger.info('Demo done.')

2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-05 16:29:33,892   INFO  -----------------Quick Demo of OpenPCDet-------------------------


cfg _file  =  /home/gx/GitHub/OpenPCDet/tools/cfgs/kitti_models/second.yaml
args =  <__main__.Args object at 0x7f8b20386710>
cfg =  {'ROOT_DIR': PosixPath('/home/gx/GitHub/OpenPCDet'), 'LOCAL_RANK': 0, 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'], 'DATA_CONFIG': {'DATASET': 'KittiDataset', 'DATA_PATH': '../data/kitti', 'POINT_CLOUD_RANGE': [0, -40, -3, 70.4, 40, 1], 'DATA_SPLIT': {'train': 'train', 'test': 'val'}, 'INFO_PATH': {'train': ['kitti_infos_train.pkl'], 'test': ['kitti_infos_val.pkl']}, 'FOV_POINTS_ONLY': True, 'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'], 'AUG_CONFIG_LIST': [{'NAME': 'gt_sampling', 'USE_ROAD_PLANE': False, 'DB_INFO_PATH': ['kitti_dbinfos_train.pkl'], 'PREPARE': {'filter_by_min_points': ['Car:5', 'Pedestrian:5', 'Cyclist:5'], 'filter_by_difficulty': [-1]}, 'SAMPLE_GROUPS': ['Car:20', 'Pedestrian:15', 'Cyclist:15'], 'NUM_POINT_FEATURES': 4, 'DATABASE_WITH_FAKELIDAR': False, 'REMOVE_EXTRA_WIDTH': [0.0, 0.0, 0.0], 'LIMIT_WHOLE_SCENE': True}, {'NAME'

2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,170   INFO  Loading KITTI dataset
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-05 16:29:34,312   INFO  Total samples for KITTI dataset: 3769
2020-08-

self.exp =  [False, False, False]
feature_map_size = [array([176, 200]), array([176, 200]), array([176, 200])]


2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,590   INFO  ==> Done (loaded 163/163)
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,615   INFO  Visualized sample index: 	1
2020-08-05 16:29:34,676   INFO  Visualized sample index:

[{'pred_boxes': tensor([[ 4.6138e+01, -4.6407e+00,  9.3130e-03,  1.6946e+00,  4.4398e-01,
          1.7678e+00,  6.4895e+00],
        [ 2.9531e+01, -7.1291e+00, -6.1918e-01,  3.4374e+00,  1.5580e+00,
          1.4106e+00,  6.0744e+00],
        [ 5.8811e+01,  1.6704e+01, -8.7981e-01,  3.8522e+00,  1.6228e+00,
          1.5095e+00,  3.3363e+00],
        [ 4.7970e+01,  2.5869e+01, -4.6201e-01,  1.5563e+00,  3.4861e-01,
          1.6139e+00,  6.8675e+00],
        [ 1.2720e+01, -8.1863e+00, -4.8355e-01,  3.9285e-01,  5.5719e-01,
          1.7558e+00,  4.5747e+00],
        [ 5.0739e+01,  2.3595e+01, -4.1340e-01,  4.1684e+00,  1.7023e+00,
          1.5655e+00,  3.0532e+00],
        [ 2.7422e+01, -1.0683e+01, -3.0245e-01,  3.6374e+00,  1.5771e+00,
          1.4566e+00,  6.5398e+00],
        [ 3.3381e+01,  2.4682e+01, -8.9336e-01,  1.7767e+00,  4.9178e-01,
          1.7044e+00,  3.5992e+00],
        [ 2.9004e+01,  2.6117e+01, -1.4427e+00,  3.6977e+00,  1.7056e+00,
          1.4859e+00,  1.9075e

In [42]:

demo_dataset.get_image_shape('000003')


array([ 375, 1242], dtype=int32)

In [43]:
demo_dataset[2]

{'points': array([[ 5.7657e+01,  2.0996e+01,  2.2840e+00,  0.0000e+00],
        [ 5.7602e+01,  2.1181e+01,  2.2840e+00,  0.0000e+00],
        [ 5.7344e+01,  2.1702e+01,  2.2820e+00,  0.0000e+00],
        ...,
        [ 6.3940e+00, -4.2000e-02, -1.6710e+00,  1.8000e-01],
        [ 6.3900e+00, -2.2000e-02, -1.6700e+00,  2.6000e-01],
        [ 6.3900e+00, -2.0000e-03, -1.6700e+00,  2.3000e-01]],
       dtype=float32),
 'frame_id': '000004',
 'calib': <pcdet.utils.calibration_kitti.Calibration at 0x7f8b2e0ae310>,
 'gt_boxes': array([[38.549694 , 15.734734 , -0.9212283,  4.01     ,  1.76     ,
          1.49     , -3.1407964,  1.       ],
        [51.45968  , 15.9170685, -0.9093928,  3.41     ,  1.8      ,
          1.38     , -3.1507964,  1.       ]], dtype=float32),
 'use_lead_xyz': True,
 'voxels': array([[[ 1.7291e+01, -1.3638e+01,  9.5400e-01,  1.5000e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
    

In [39]:
demo_dataset[13]['gt_boxes']

array([[ 7.3751190e+01,  1.1799237e+00, -8.2214653e-01,  3.5599999e+00,
         1.6000000e+00,  1.2600000e+00,  1.9203663e-02,  1.0000000e+00]],
      dtype=float32)

In [35]:
data_dict['gt_boxes'].shape

(1, 8)

In [30]:
pred_dicts[0]

{'pred_boxes': tensor([[ 38.4499,  15.7031,  -0.9063,   3.7692,   1.7011,   1.4608,   3.1156],
         [ 51.5840,  15.9201,  -0.8220,   3.8331,   1.6007,   1.4393,   3.1067],
         [ 58.6575,  19.6187,  -0.3599,   1.8299,   0.5985,   1.7520,   3.3225],
         [ 33.0857, -28.4273,  -0.3681,   0.7259,   0.6675,   1.8104,   5.7163],
         [ 17.6305, -13.3460,  -0.1972,   0.8061,   0.6008,   1.8643,   3.0827],
         [ 43.4791,  30.9913,  -0.6152,   4.2695,   1.6868,   1.5767,   5.1174],
         [ 48.1766,  22.6731,  -0.7106,   1.8919,   0.5208,   1.7132,   3.4704],
         [ 17.6793, -12.5326,  -0.4277,   0.6042,   0.4929,   1.7065,   0.9882],
         [ 38.0470, -10.9097,  -0.6043,   4.1228,   1.6092,   1.5099,   2.6666],
         [ 37.7178,  27.1945,  -1.3464,   3.4665,   1.7198,   1.4048,   2.9767],
         [ 70.3897,  18.3060,   0.1208,   3.7806,   1.6709,   1.6583,   6.4339],
         [ 47.0214, -10.2587,  -0.5924,   1.5295,   0.3279,   1.6574,   5.9087],
         [ 20.

In [31]:
ret_dict

{'gt': 2,
 'roi_0.3': 0,
 'rcnn_0.3': 2,
 'roi_0.5': 0,
 'rcnn_0.5': 2,
 'roi_0.7': 0,
 'rcnn_0.7': 2}

In [32]:
annos

[{'name': array(['Car', 'Car', 'Cyclist', 'Pedestrian', 'Pedestrian', 'Car',
         'Cyclist', 'Pedestrian', 'Car', 'Car', 'Car', 'Cyclist',
         'Pedestrian'], dtype='<U10'),
  'truncated': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'occluded': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'alpha': array([-4.2986536, -4.378186 , -4.5705433, -7.9969497, -5.301493 ,
         -6.0689316, -4.6013026, -3.175675 , -4.5166264, -3.922865 ,
         -7.7502356, -7.694255 , -4.6783867], dtype=float32),
  'bbox': array([[ 281.2606  ,  184.89514 ,  344.2302  ,  215.23387 ],
         [ 365.70273 ,  182.8096  ,  406.52835 ,  204.63632 ],
         [ 362.48904 ,  175.64127 ,  373.58954 ,  197.7507  ],
         [1225.715   ,  160.20007 , 1241.      ,  200.62897 ],
         [1143.4647  ,  140.16597 , 1193.5139  ,  219.64955 ],
         [  44.94606 ,  181.4463  ,  138.16238 ,  209.4764  ],
         [ 263.07816 ,  180.56128 ,  275.2236  ,  207.23495 ],
         

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def visualize_pts(pts, fig=None, bgcolor=(1,1,1), fgcolor=(1.0, 1.0, 1.0),
                  show_intensity=True, size=(600, 600), draw_origin=True):
    if not isinstance(pts, np.ndarray):
        pts = pts.cpu().numpy()
    if fig is None:
        fig = plt.figure(figsize=(20, 10))
        
        ax = fig.add_subplot(111, projection='3d')
        ax.set_facecolor(bgcolor)

    if show_intensity:

        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], c=pts[:,3], s=0.01, marker=',')
    else:
        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], s=0.01, marker=',')
        
    if draw_origin:
        ax.plot([0,3], [0,0], [0,0], color=(0,0,1))
        ax.plot([0,0], [0,3], [0,0], color=(0,1,0))
        ax.plot([0,0], [0,0], [0,3], color=(1,0,0))
    
    return ax

    # if draw_origin:

In [ ]:
fig = plt.figure()
points = data_dict['points']

plt.plot(points[:,2])
fig = plt.figure()
plt.plot(points[:,1])
fig = plt.figure()
plt.plot(points[:,0])

In [131]:
ax = visualize_pts(points)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
def draw_grid(x1, y1, x2, y2, ax, color=(1,0.5,0.5)):
    ax.plot([x1, x1], [y1, y2], [0, 0], color=color, linewidth=1)
    ax.plot([x2, x2], [y1, y2], [0, 0], color=color, linewidth=1)
    ax.plot([x1, x2], [y1, y1], [0, 0], color=color, linewidth=1)
    ax.plot([x1, x2], [y2, y2], [0, 0], color=color, linewidth=1)
    return ax

def draw_multi_grid_range(ax, grid_size=20, bv_range=(-60, -60, 60, 60)):
    for x in range(bv_range[0], bv_range[2], grid_size):
        for y in range(bv_range[1], bv_range[3], grid_size):
            ax = draw_grid(x, y, x + grid_size, y+grid_size, ax) 
    return ax

In [133]:
draw_multi_grid_range(ax, bv_range=(0, -40, 80, 40))

<Axes3DSubplot:>

In [134]:
def check_numpy_to_torch(x):
    if isinstance(x, np.ndarray):
        return torch.from_numpy(x).float(), True
    return x, False


def rotate_points_along_z(points, angle):
    """
    Args:
        points: (B, N, 3 + C)
        angle: (B), angle along z-axis, angle increases x ==> y
    Returns:

    """
    points, is_numpy = check_numpy_to_torch(points)
    angle, _ = check_numpy_to_torch(angle)

    cosa = torch.cos(angle)
    sina = torch.sin(angle)
    zeros = angle.new_zeros(points.shape[0])
    ones = angle.new_ones(points.shape[0])
    rot_matrix = torch.stack((
        cosa,  sina, zeros,
        -sina, cosa, zeros,
        zeros, zeros, ones
    ), dim=1).view(-1, 3, 3).float()
    points_rot = torch.matmul(points[:, :, 0:3], rot_matrix)
    points_rot = torch.cat((points_rot, points[:, :, 3:]), dim=-1)
    return points_rot.numpy() if is_numpy else points_rot


def boxes_to_corners_3d(boxes3d):
    """
        7 -------- 4
       /|         /|
      6 -------- 5 .
      | |        | |
      . 3 -------- 0
      |/         |/
      2 -------- 1
    Args:
        boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center

    Returns:
    """
    boxes3d, is_numpy = check_numpy_to_torch(boxes3d)

    template = boxes3d.new_tensor((
        [1, 1, -1], [1, -1, -1], [-1, -1, -1], [-1, 1, -1],
        [1, 1, 1], [1, -1, 1], [-1, -1, 1], [-1, 1, 1],
    )) / 2

    corners3d = boxes3d[:, None, 3:6].repeat(1, 8, 1) * template[None, :, :]
    corners3d = rotate_points_along_z(corners3d.view(-1, 8, 3), boxes3d[:, 6]).view(-1, 8, 3)
    corners3d += boxes3d[:, None, 0:3]

    return corners3d.numpy() if is_numpy else corners3d

In [135]:

def draw_corners3d(corners3d, ax, color=(1, 1, 1), line_width=1, cls=None, tag='', max_num=500, tube_radius=None):
    """
    :param corners3d: (N, 8, 3)
    :param fig:
    :param color:
    :param line_width:
    :param cls:
    :param tag:
    :param max_num:
    :return:
    """
    num = min(max_num, len(corners3d))
    for n in range(num):
        b = corners3d[n]  # (8, 3)

        if cls is not None:
            if isinstance(cls, np.ndarray):
                ax.text(b[6, 0], b[6, 1], b[6, 2], '%.2f' % cls[n], fontsize=0.3, color=color)
            else:
                ax.text(b[6, 0], b[6, 1], b[6, 2], '%s' % cls[n], fontsize=0.3, color=color)

        for k in range(0, 4):
            i, j = k, (k + 1) % 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

            i, j = k + 4, (k + 1) % 4 + 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

            i, j = k, k + 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

        i, j = 0, 5
        ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                    linewidth=line_width)
        i, j = 1, 4
        ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                    linewidth=line_width)

    return ax



In [136]:
ref_boxes=pred_dicts[0]['pred_boxes'].cpu().numpy()
ref_scores=pred_dicts[0]['pred_scores'].cpu().numpy()
ref_labels=pred_dicts[0]['pred_labels'].cpu().numpy()

ref_corners3d = boxes_to_corners_3d(ref_boxes)
ax = draw_corners3d(ref_corners3d, ax, color=(0,1,0), cls=ref_scores, max_num=100)

box_colormap = [
    [1, 1, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 1, 0],
]

gt_boxes = data_dict['gt_boxes']
if gt_boxes is not None:
    corners3d = boxes_to_corners_3d(gt_boxes)
    ax = draw_corners3d(corners3d, ax, color=(0, 0, 1), max_num=100)

    if ref_boxes is not None:
        ref_corners3d = boxes_to_corners_3d(ref_boxes)
        if ref_labels is None:
            ax = draw_corners3d(ref_corners3d, ax, color=(0, 1, 0), cls=ref_scores, max_num=100)
        else:
            for k in range(ref_labels.min(), ref_labels.max() + 1):
                cur_color = tuple(box_colormap[k % len(box_colormap)])
                mask = (ref_labels == k)
                ax = draw_corners3d(ref_corners3d[mask], ax, color=cur_color, cls=ref_scores[mask], max_num=100)

In [137]:
data_dict.keys()

dict_keys(['points', 'frame_id', 'gt_boxes', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'image_shape'])